In [193]:
import spacy
from spacy.tokens import Span # Doc, Span, Token
import textacy
from event2mind_hack import load_event2mind_archive
from allennlp.predictors.predictor import Predictor
import math
import re

# Span.set_extension('is_entity', default=False, force=True)
Span.set_extension('entity_id', default=None, force=True)
# Token.set_extension('in_entity', default=False, force=True)
# Token.set_extension('entity_ids', default=[], force=True)
# def get_entity_ids()
# Token.set_extension('entity_ids', getter=get_entity_ids, setter=set_entity_ids)
# assert 

archive = load_event2mind_archive('data/event2mind.tar.gz')
event2mind_predictor = Predictor.from_archive(archive)
            
print('loading en_coref_md...')
nlp = spacy.load('en_coref_md')
print('done')

class Entity:
    def __init__(self, id_, text, class_):
        self.id = id_
        self.text = text
        self.class_ = class_

class Statement:
    def __init__(self, id_, subject_text, predicate_text, subject_id=None, object_text=None, object_id=None, weight=None):
        self.id = id_
        self.subject_text = subject_text
        self.subject_id = subject_id
        self.predicate_text = predicate_text
        self.object_text = object_text
        self.object_id = object_id
        self.weight = weight

# memoryless for now; each change to text means a whole new model
class Model:
    def __init__(self, text):
        self.raw = text
        preprocessed = textacy.preprocess.normalize_whitespace(text)
        preprocessed = textacy.preprocess.fix_bad_unicode(preprocessed)
        preprocessed = textacy.preprocess.unpack_contractions(preprocessed)
        preprocessed = textacy.preprocess.remove_accents(preprocessed)
#         preprocessed = textacy.preprocess.preprocess_text(preprocessed, fix_unicode=True, no_contractions=True, no_accents=True)
        self.doc = nlp(preprocessed)
        self.named_entities = [ent for ent in self.doc.ents]
        self.noun_chunks = [chunk for chunk in self.doc.noun_chunks]

#         self.resolved_text = self.get_resolved_text()
#         self.resolved_doc = nlp(self.resolved_text)

        self.user = None
        self.entities = []
        self.extract_user_entity()
        self.extract_coref_entities()
        
        # create entities for the rest of the noun chunks
        for noun_chunk in self.noun_chunks:
            if noun_chunk._.entity_id == None:
                entity = Entity(id_=len(self.entities), text=noun_chunk.text, class_='THING')
                self.entities.append(entity)
                noun_chunk._.entity_id = entity.id
        
        # assumes all subjects come before verbs...or at least, only extracts where true
        self.statements = []
        for sent in self.doc.sents:
            self.extract_statements(sent)
        self.generate_event2mind_statements()
                

    def get_entity_refs(self, entity_id):
        entity_refs = []
        for nc in self.noun_chunks:
            if nc._.entity_id == entity_id:
                entity.refs.append(nc)
        return entity_refs
    
    def get_entity(self, entity_id):
        for entity in self.entities:
            if entity.id == entity_id:
                return entity
        return None
    
    def get_statement(self, statement_id):
        for statement in self.statements:
            if statement.id == statement_id:
                return statement
        return None
    
    def extract_user_entity(self):
        text = 'User'
        class_ = 'PERSON'
        entity = Entity(id_=len(self.entities), text = text, class_=class_)
        self.user = entity
        self.entities.append(entity)
        
        for chunk in self.doc.noun_chunks:
            if chunk.root.text.lower() in ['i', 'me', 'myself']:
#                 chunk._.is_entity = True
                chunk._.entity_id = entity.id
#                 for token in chunk:
#                     token._.in_entity = True
#                     token._.entity_ids.append(entity.id)
            
            # TODO: catch possessives, create 'belongs to' & entity
    
    def extract_coref_entities(self):
        def get_match(span_to_match, spans):
            for span in spans:
                if span.root.i == span_to_match.root.i:
                    return span # TODO: what if there are somehow multiple named entities with same root as mention?
            return None
                
        for cluster in self.doc._.coref_clusters:
            
            # find named entity w same root as main mention
            entity_ref = get_match(cluster.main, self.named_entities)
            
            # else find named entity w same root as any mention
            if entity_ref == None:
                for mention in cluster.mentions:
                    entity_ref = get_match(mention, self.named_entities) # TODO: what if different mentions have different entities?
            
            # else use main mention for named entity match 
            if entity_ref == None:
                entity_ref = cluster.main
            
            # don't work with corefs that are user or don't have noun chunk
            # TODO: is it dumb to not work with corefs that don't have noun chunk?
            not_user = entity_ref.root.text.lower() not in ['i', 'me', 'myself'] #TODO: replace with something that doesn't allow duplicate entities (unless I want them order independent?)            
            nc_match = get_match(entity_ref, self.noun_chunks) # TODO: if no nc matched it might be possessive or something, in which case add that then 'belong to'
            if not_user and nc_match != None:
                text = entity_ref
                if entity_ref.label_ == '':
                    class_ = 'THING'
                else:
                    class_ = entity_ref.label_
                    
                id_ = len(self.entities)
                entity = Entity(id_=id_, text = text, class_=class_)
                self.entities.append(entity)

                # TODO: I'm setting some of these twice - is that bad?
                for mention in cluster.mentions:
                    if mention.root.pos_ in ['NOUN', 'PROPN', 'PRON']:
                        
                        # only set extension for mentions with noun chunk matches
                        # TODO: is it dumb to set extension for noun chunk, not direct? it's usually the same..when is it not?
                        nc_match = get_match(mention, self.noun_chunks)
                        if nc_match != None:
                            nc_match._.entity_id = entity.id
    
    # TODO: needs fine tuning, strips a lot out and adds some that shouldn't be
    def extract_statements(self, span, previous_subject=None, previous_predicate=None):
        # TODO: I think sometimes conjunctions are missing predicates? maybe not
        first, last = textacy.spacier.utils.get_span_for_verb_auxiliaries(span.root)
        beginning = self.doc[span.start:first]
        middle = self.doc[first:last+1]
        end = self.doc[last+1:span.end]

        # if beginning has nsubj token and that token is in a noun chunk, set noun chunk as subject entity
        # assumes there aren't two nsubj in beginning, safe?
        subject = None
        for token in beginning:
            if token.dep_ == 'nsubj':
                for noun_chunk in self.noun_chunks:
                    if noun_chunk.root == token:
                        subject = noun_chunk
                        break
                else:
                    # so that every subject has an entity
                    # TODO: but not all objects have entities
                    entity = Entity(id_=len(self.entities), text = token.text, class_=None)
                    self.entities.append(entity)
                    subject = self.doc[token.i:token.i+1]
                    break
        

        # TODO: just use it if you can't find noun chunk

        # if this is being called on remainder (conjunction) with no subject, use previous subject
        # eg 'I need to drink, and to drink': 'and to eat' borrows 'I' for subject
        if subject == None and previous_subject != None:
            subject = previous_subject

        # TODO: process the verb phrase a little to consolidate statements
        # use textacy's consolidate?
        predicate = middle
        print(predicate.text)

        conjuncted = None
        for token in end:
            if token.dep_ == 'conj':
                object_ = self.doc[last+1:token.left_edge.i]
                conjuncted = self.doc[token.left_edge.i:token.right_edge.i+1]
                break
        else:
            object_ = end
            
        while self.doc[object_.end-1].dep_ in ['cc', 'punct']:
            object_ = self.doc[object_.start:object_.end-1]
        
        print(span)
        print(subject, '-', predicate, '-', object_)
        print()
        
        if subject != None and predicate != None:
            if predicate.text == '\'s':
                predicate_text = 'is'
            else:
                predicate_text = predicate.text
            if object_ == None:
                object_text = None
            else:
                object_text = object_.text
            statement = Statement(
                id_=len(self.statements),
                subject_text = subject.text,
                subject_id = subject._.entity_id, # sometimes None, is that bad form?
                predicate_text = predicate.text,
                object_text = object_text,
                object_id = None, # for now, later might extract entity from subject phrase
                weight = None
            )
            self.statements.append(statement)
        
        if conjuncted != None:
            self.extract_statements(conjuncted, previous_subject=subject)

    def generate_event2mind_statements(self):
        # TODO: sub PersonY for people in subject
        # TODO: needs refactoring generally (everything does..)
        # TODO: the differening text all over the place is a nightmare, fix

        # for turning emotion string texts into proper emotion strings
        def str_to_list(emotion_str):
            emotion_str = emotion_str[2:-2]
            if emotion_str[-1] == '.':
                emotion_str = emotion_str[:-1]
            return re.split('","|", "|, ',emotion_str)
        
        person_statements = []
        # TODO: with a db this would be an sql query or something
        for statement in self.statements:
             if statement.subject_id != None:
                subject_entity = self.get_entity(statement.subject_id)
                if subject_entity != None and subject_entity.class_ == 'PERSON':
                    person_statements.append(statement)
        
        for statement in person_statements:
            prediction = event2mind_predictor.predict(
              source='PersonX ' + str(statement.predicate_text) + ' ' + str(statement.object_text)
            )

            for emotion, log_p in zip(
                prediction['xreact_top_k_predicted_tokens'],
                prediction['xreact_top_k_log_probabilities']
            ):
                emotion = ' '.join(emotion)
                p = math.exp(log_p)
                feels_statement = Statement(
                    id_=len(self.statements),
                    subject_text = subject_entity.text,
                    subject_id = subject_entity.id,
                    predicate_text = 'feels',
                    object_text = emotion,
                    object_id = None,
                    weight = p
                )
                print(statement.id, statement.subject_text, statement.predicate_text, statement.object_text)
                print(feels_statement.id, feels_statement.subject_text, feels_statement.predicate_text, feels_statement.object_text, round(feels_statement.weight, 2))
                self.statements.append(feels_statement)
            
#     def get_resolved_text(self):
#         resolved_text = [token.text_with_ws for token in self.doc]
#         for person in self.people:
#             for ref in person.refs:

#                 # determine resolved value
#                 # resolved_value = '[' + person.name.upper() + ']'
#                 resolved_value = person.name.upper()
#                 if ref.root.tag_ == 'PRP$':
#                     resolved_value += '\'s'
#                 if ref.text_with_ws[-1] == ' ':
#                     resolved_value += ' '

#                 # set first token to value, remaining tokens to ''
#                 resolved_text[ref.start] = resolved_value
#                 for i in range(ref.start+1, ref.end):
#                     resolved_text[i] = ''
#         return ''.join(resolved_text)

loading en_coref_md...
done


In [203]:
doc3 = nlp('Jason went to the store. I like John. Mary and Sue walked to the beach and both got ice cream.')
print(len([ent for ent in doc3.ents]))
print([ent for ent in doc3.ents])

4
[Jason, John, Mary, Sue]


In [194]:
text = "Sheila was run over by a truck. She herself didn't see that coming. I told her she should take care of herself, but I know she'll just go and do her thing regardless of what I say. What a conundrum! This makes me wish I had never signed up to be friends with her, although I do love the girl."

In [195]:
text = 'She washed her truck yesterday. Sheila said it doesn\'t look that great but she herself had to say she really impressed herself'

In [196]:
text = 'So I have had a good day today. I found out we got the other half of our funding for my travel grant, which paid for my friend to come with me. So that’s good, she and I will both get some money back. I took my dogs to the pet store so my girl dog could get a new collar, but she wanted to beat everyone up. This is an ongoing issue with her. She’s so little and cute too but damn she acts like she’s gonna go for the jugular with everyone she doesn’t know! She did end up with a cute new collar tho, it has pineapples on it. I went to the dentist and she’s happy with my Invisalign progress. We have three more trays and then she does an impression to make sure my teeth are where they need to be before they get the rest of the trays. YAY! And I don’t have to make another payment until closer to the end of my treatment. I had some work emails with the festival, and Jessie was bringing up some important points, and one of our potential artists was too expensive to work with, so Mutual Friend was asking for names for some other people we could work with. So I suggested like, three artists, and Jessie actually liked the idea of one of them doing it. Which is nice. I notice she is very encouraging at whatever I contribute to our collective. It’s sweet. I kind of know this is like, the only link we have with each other right now besides social media, so it seems like she’s trying to make sure I know she still wants me to be involved and doesn’t have bad feelings for me. And there was a short period when I was seriously thinking of leaving the collective and not working with this festival anymore. I was so sad, and felt so upset, and didn’t know what to do about Jessie. It felt really close to me throwing in the towel. But I hung on through the festival and it doesn’t seem so bad from this viewpoint now with more time that has passed. And we have been gentle, if reserved, with each other. I mean her last personal email to me however many weeks ago wasn’t very nice. But it seems like we’ve been able to put it aside for work reasons. I dunno. I still feel like if anything was gonna get mended between us, she would need to make the first moves on that. I really don’t want to try reaching out and get rejected even as a friend again. I miss her though. And sometimes I think she misses me. But I don’t want to approach her assuming we both miss each other and have her turn it on me again and make out like all these things are all in my head. I don’t know about that butch I went on a date with last night. I feel more of a friend vibe from her, than a romantic one. I can’t help it, I am just not attracted to butches. And I don’t know how to flirt with them. And I don’t think of them in a sexy way. But I WOULD like another butch buddy. I mean yeah maybe Femmes do play games, or maybe I just chased all the wrong Femmes. Maybe I’ll just leave this and not think about it much until I get back to town in January.'

In [197]:
model = Model(text)

have had
So I have had a good day today.
I - have had - a good day today

found
I found out we got the other half of our funding for my travel grant, which paid for my friend to come with me.
I - found - out we got the other half of our funding for my travel grant, which paid for my friend to come with me

get
So that's good, she and I will both get some money back.
that - get - some money back

took
I took my dogs to the pet store so my girl dog could get a new collar, but she wanted to beat everyone up.
I - took - my dogs to the pet store so my girl dog could get a new collar

wanted
she wanted to beat everyone up.
she - wanted - to beat everyone up

is
This is an ongoing issue with her.
This - is - an ongoing issue with her

's
She's so little and cute too
She - 's - so little

cute
cute
She - cute - 

acts
but damn she acts like she's gonna go for the jugular with everyone
she - acts - like she's gonna go for the jugular with everyone

does not know
she does not know!
she - does no

12 I went to the dentist
89 User feels relieved 0.15
12 I went to the dentist
90 User feels satisfied 0.05
12 I went to the dentist
91 User feels happy 0.04
12 I went to the dentist
92 User feels accomplished 0.04
12 I went to the dentist
93 User feels worried 0.04
12 I went to the dentist
94 User feels better 0.03
12 I went to the dentist
95 User feels nervous 0.03
12 I went to the dentist
96 User feels responsible 0.02
12 I went to the dentist
97 User feels proud 0.02
12 I went to the dentist
98 User feels scared 0.02
16 I do not have to make another payment until closer to the end of my treatment
99 User feels sad 0.08
16 I do not have to make another payment until closer to the end of my treatment
100 User feels worried 0.08
16 I do not have to make another payment until closer to the end of my treatment
101 User feels upset 0.05
16 I do not have to make another payment until closer to the end of my treatment
102 User feels sick 0.03
16 I do not have to make another payment until c

230 User feels satisfied 0.05
38 I dunno 
231 User feels accomplished 0.05
38 I dunno 
232 User feels nervous 0.04
38 I dunno 
233 User feels proud 0.03
38 I dunno 
234 User feels excited 0.02
38 I dunno 
235 User feels relieved 0.02
38 I dunno 
236 User feels determined 0.02
38 I dunno 
237 User feels scared 0.02
38 I dunno 
238 User feels like a good parent 0.0
39 I feel like if anything was gonna get mended between us, she would need to make the first moves on that
239 User feels sad 0.04
39 I feel like if anything was gonna get mended between us, she would need to make the first moves on that
240 User feels scared 0.04
39 I feel like if anything was gonna get mended between us, she would need to make the first moves on that
241 User feels happy 0.04
39 I feel like if anything was gonna get mended between us, she would need to make the first moves on that
242 User feels upset 0.04
39 I feel like if anything was gonna get mended between us, she would need to make the first moves on t

51 I do not know how to flirt with them
359 User feels upset 0.06
51 I do not know how to flirt with them
360 User feels disappointed 0.04
51 I do not know how to flirt with them
361 User feels sad 0.03
51 I do not know how to flirt with them
362 User feels worried 0.03
51 I do not know how to flirt with them
363 User feels frustrated 0.03
51 I do not know how to flirt with them
364 User feels annoyed 0.02
51 I do not know how to flirt with them
365 User feels unhappy 0.02
51 I do not know how to flirt with them
366 User feels angry 0.02
51 I do not know how to flirt with them
367 User feels anxious 0.02
51 I do not know how to flirt with them
368 User feels confused 0.02
52 I do not think of them in a sexy way
369 User feels upset 0.04
52 I do not think of them in a sexy way
370 User feels disappointed 0.04
52 I do not think of them in a sexy way
371 User feels nervous 0.02
52 I do not think of them in a sexy way
372 User feels sad 0.02
52 I do not think of them in a sexy way
373 User

In [192]:
     event2mind_predictor.predict(
              source="PersonX drops a hint"
            )

{'xintent_top_k_predictions': [[4, 3, 3, 3],
  [684, 3, 3, 3],
  [323, 3, 3, 3],
  [282, 3, 3, 3],
  [255, 3, 3, 3],
  [153, 229, 3, 3],
  [13, 175, 3, 3],
  [44, 251, 3, 3],
  [13, 267, 3, 3],
  [211, 32, 80, 3]],
 'xintent_top_k_log_probabilities': [-1.128340721130371,
  -4.298313617706299,
  -4.499514579772949,
  -4.616389751434326,
  -4.652942657470703,
  -5.256786823272705,
  -5.543419361114502,
  -6.178347587585449,
  -6.431229114532471,
  -9.508707046508789],
 'xreact_top_k_predictions': [[54, 3],
  [70, 3],
  [53, 3],
  [109, 3],
  [5, 3],
  [73, 3],
  [11, 3],
  [92, 3],
  [25, 3],
  [63, 3]],
 'xreact_top_k_log_probabilities': [-3.147449016571045,
  -3.159241199493408,
  -3.1768059730529785,
  -3.2743079662323,
  -3.3373990058898926,
  -3.516559362411499,
  -3.596619129180908,
  -3.8112082481384277,
  -3.8679120540618896,
  -3.9374184608459473],
 'oreact_top_k_predictions': [[4, 3],
  [63, 3],
  [36, 3],
  [89, 3],
  [83, 3],
  [91, 3],
  [138, 3],
  [53, 3],
  [92, 3],
  [5,

In [96]:
for statement in model.statements:
    if statement.subject_id == model.user.id:
        print(statement.subject_text, statement.predicate_text, statement.object_text)

I have had a good day today
I found out we got the other half of our funding for my travel grant, which paid for my friend to come with me
I took my dogs to the pet store so my girl dog could get a new collar
I went to the dentist
I do not have to make another payment until closer to the end of my treatment
I had some work emails with the festival
I suggested like, three artists
I notice she is very encouraging at whatever I contribute to our collective
I was so sad
I felt so upset
I did not know what to do about Jessie
I hung on through the festival
I mean her last personal email to me
I dunno 
I feel like if anything was gonna get mended between us, she would need to make the first moves on that
I do not want to try reaching out
I get rejected even as a friend again
I miss her though
I think she misses me
I do not want to approach her assuming we both miss each other
I have her turn it on me again
I make out like all these things are all in my head
I do not know about that butch
I we

In [61]:
doc = nlp('I need to pee.')
for sent in doc.sents:
    print(doc[sent.start:sent.end])

I need to pee.


In [235]:
def get_match(span_to_match, spans):
    for span in spans:
        if span.root.i == span_to_match.root.i:
            return span # TODO: what if there are somehow multiple named entities with same root as mention?
    return None

# assumes all subjects come before verbs...or at least, only extracts where true
for sent in model.doc.sents:
    first, last = textacy.spacier.utils.get_span_for_verb_auxiliaries(sent.root)
    beginning = model.doc[sent.start:first]
    middle = model.doc[first:last+1]
    end = model.doc[last+1:sent.end]
    
    # if beginning has nsubj token and that token is in a noun chunk, set noun chunk as subject entity
    # assumes there aren't two nsubj in beginning, safe?
    entity_1 = None
    for token in beginning:
        if token.dep_ == 'nsubj':
            for noun_chunk in model.noun_chunks:
                if noun_chunk.root == token:
                    entity_1 = noun_chunk
                    break
    
    # TODO: process the verb phrase a little to consolidate statements
    # use textacy's consolidate?
    relation = middle
    
    # TODO: kill trailing punct and cc
    remainder = None
    for token in end:
        if token.dep_ == 'conj':
            entity_2 = model.doc[last+1:token.left_edge.i]
            remainder = model.doc[token.left_edge.i:token.right_edge.i+1]
            break
    else:
        entity_2 = end
    
    print(entity_1, '-', relation, '-', entity_2, '-', remainder)
    

I - have had - a good day today. - None
I - found - out we got the other half of our funding for my travel grant, which paid for my friend to come with me. - None
she - get - some money back. - None
I - took - my dogs to the pet store so my girl dog could get a new collar, but - she wanted to beat everyone up.
None - is - an ongoing issue with her. - None
She - 's - so little and - cute
she - acts - like she's gonna go for the jugular with everyone - None
she - does not know - ! - None
She - did end - up with a cute new collar tho - None
it - has - pineapples on it. - None
I - went - to the dentist - None
she - 's - happy with my Invisalign progress. - None
We - have - three more trays - None
she - does - an impression to make sure my teeth are where they need to be before they get the rest of the trays. - None
None - YAY - ! - None
I - do not have - to make another payment until closer to the end of my treatment. - None
I - had - some work emails with the festival, and - Jessie was br

In [237]:
type(model.doc[0:0])

spacy.tokens.span.Span

In [233]:
for token in model.doc:
    print(token.dep_)

advmod
nsubj
aux
ROOT
det
amod
dobj
npadvmod
punct
nsubj
ROOT
prt
nsubj
ccomp
det
amod
dobj
prep
poss
pobj
prep
poss
compound
pobj
punct
nsubj
relcl
prep
poss
pobj
aux
advcl
prep
pobj
punct
advmod
nsubj
advcl
acomp
punct
nsubj
cc
conj
aux
advmod
ROOT
det
dobj
advmod
punct
nsubj
ROOT
poss
dobj
prep
det
amod
pobj
mark
poss
compound
nsubj
aux
advcl
det
amod
dobj
punct
cc
nsubj
conj
aux
xcomp
dobj
prt
punct
nsubj
ROOT
det
amod
attr
prep
pobj
punct
nsubj
ROOT
advmod
acomp
cc
conj
advmod
cc
intj
nsubj
ROOT
mark
nsubj
aux
advcl
aux
xcomp
prep
det
pobj
prep
pobj
nsubj
aux
neg
ROOT
punct
nsubj
aux
ROOT
prt
prep
det
amod
amod
compound
pobj
punct
nsubj
ROOT
dobj
prep
pobj
punct
nsubj
ROOT
prep
det
pobj
cc
nsubj
ROOT
acomp
prep
poss
compound
pobj
punct
nsubj
ROOT
nummod
amod
dobj
cc
advmod
nsubj
ROOT
det
dobj
aux
acl
acomp
poss
nsubj
ccomp
advmod
nsubj
advcl
aux
xcomp
mark
nsubj
advcl
det
dobj
prep
det
pobj
punct
ROOT
punct
cc
nsubj
aux
neg
ROOT
aux
xcomp
det
dobj
prep
pcomp
prep
det
pobj
prep
pos

In [221]:
for sent in model.doc.sents:
    first, last = textacy.spacier.utils.get_span_for_verb_auxiliaries(sent.root)
    print('[', sent, ']', model.doc[first:last+1])

[ So I have had a good day today. ] have had
[ I found out we got the other half of our funding for my travel grant, which paid for my friend to come with me. ] found
[ So that's good, she and I will both get some money back. ] get
[ I took my dogs to the pet store so my girl dog could get a new collar, but she wanted to beat everyone up. ] took
[ This is an ongoing issue with her. ] is
[ She's so little and cute too ] 's
[ but damn she acts like she's gonna go for the jugular with everyone ] acts
[ she does not know! ] does not know
[ She did end up with a cute new collar tho ] did end
[ , it has pineapples on it. ] has
[ I went to the dentist ] went
[ and she's happy with my Invisalign progress. ] 's
[ We have three more trays ] have
[ and then she does an impression to make sure my teeth are where they need to be before they get the rest of the trays. ] does
[ YAY! ] YAY
[ And I do not have to make another payment until closer to the end of my treatment. ] do not have
[ I had some w

In [211]:
for noun_chunk in model.noun_chunks:
    print(noun_chunk, noun_chunk.root.head.tag_, noun_chunk.root.head.pos_, noun_chunk.root.dep_)

I VBN VERB nsubj
a good day VBN VERB dobj
I VBD VERB nsubj
we VBD VERB nsubj
the other half VBD VERB dobj
our funding IN ADP pobj
my travel grant IN ADP pobj
my friend IN ADP pobj
me IN ADP pobj
she VB VERB nsubj
I PRP PRON conj
some money VB VERB dobj
I VBD VERB nsubj
my dogs VBD VERB dobj
the pet store IN ADP pobj
my girl dog VB VERB nsubj
a new collar VB VERB dobj
she VBD VERB nsubj
everyone VB VERB dobj
an ongoing issue VBZ VERB attr
her IN ADP pobj
She VBZ VERB nsubj
she VBZ VERB nsubj
she VBG VERB nsubj
the jugular IN ADP pobj
everyone IN ADP pobj
she VB VERB nsubj
She VB VERB nsubj
a cute new collar tho IN ADP pobj
it VBZ VERB nsubj
pineapples VBZ VERB dobj
it IN ADP pobj
I VBD VERB nsubj
the dentist IN ADP pobj
she VBZ VERB nsubj
my Invisalign progress IN ADP pobj
We VBP VERB nsubj
three more trays VBP VERB dobj
she VBZ VERB nsubj
an impression VBZ VERB dobj
my teeth VBP VERB nsubj
they VBP VERB nsubj
they VBP VERB nsubj
the rest VBP VERB dobj
the trays IN ADP pobj
I VB VERB ns

In [207]:
for entity in model.entities:
    print(entity.id, entity.text, entity.class_)

0 User PERSON
1 we None
2 my friend None
3 a cute new collar tho None
4 We None
5 three more trays None
6 they None
7 Jessie PERSON
8 three artists None
9 this festival None
10 It None
11 them None
12 a good day None
13 the other half None
14 our funding None
15 my travel grant None
16 some money None
17 my dogs None
18 the pet store None
19 my girl dog None
20 a new collar None
21 everyone None
22 an ongoing issue None
23 the jugular None
24 everyone None
25 pineapples None
26 the dentist None
27 my Invisalign progress None
28 an impression None
29 my teeth None
30 the rest None
31 another payment None
32 the end None
33 my treatment None
34 some work emails None
35 the festival None
36 some important points None
37 our potential artists None
38 Mutual Friend None
39 names None
40 some other people None
41 the idea None
42 it None
43 our collective None
44 It None
45 the only link None
46 we None
47 social media None
48 it None
49 bad feelings None
50 a short period None
51 what None


In [208]:
for chunk in model.noun_chunks:
    print(chunk._.entity_id, chunk)

0 I
12 a good day
0 I
1 we
13 the other half
14 our funding
15 my travel grant
2 my friend
0 me
2 she
0 I
16 some money
0 I
17 my dogs
18 the pet store
19 my girl dog
20 a new collar
2 she
21 everyone
22 an ongoing issue
2 her
2 She
2 she
2 she
23 the jugular
24 everyone
2 she
2 She
3 a cute new collar tho
3 it
25 pineapples
3 it
0 I
26 the dentist
2 she
27 my Invisalign progress
4 We
5 three more trays
2 she
28 an impression
29 my teeth
6 they
6 they
30 the rest
5 the trays
0 I
31 another payment
32 the end
33 my treatment
0 I
34 some work emails
35 the festival
7 Jessie
36 some important points
37 our potential artists
38 Mutual Friend
39 names
40 some other people
4 we
0 I
8 three artists
7 Jessie
41 the idea
8 them
42 it
0 I
7 she
0 I
43 our collective
44 It
0 I
45 the only link
46 we
47 social media
48 it
7 she
0 I
7 she
0 me
49 bad feelings
0 me
50 a short period
0 I
9 this festival
0 I
51 what
52 Jessie
10 It
0 me
53 the towel
0 I
9 the festival
10 it
54 this viewpoint
55 more t

In [153]:
model.doc.text

'She washed her truck yesterday. Sheila said it does not look that great but she herself had to say she really impressed herself'

In [154]:
for cluster in model.doc._.coref_clusters:
    print(cluster)

Sheila: [She, her, Sheila, she, she herself, she, herself]


In [147]:
for chunk in model.doc.noun_chunks:
    print(chunk)

I
a good day
I
we
the other half
our funding
my travel grant
my friend
me
she
I
some money
I
my dogs
the pet store
my girl dog
a new collar
she
everyone
an ongoing issue
her
She
she
she
the jugular
everyone
she
She
a cute new collar tho
it
pineapples
it
I
the dentist
she
my Invisalign progress
We
three more trays
she
an impression
my teeth
they
they
the rest
the trays
I
another payment
the end
my treatment
I
some work emails
the festival
Jessie
some important points
our potential artists
Mutual Friend
names
some other people
we
I
three artists
Jessie
the idea
them
it
I
she
I
our collective
It
I
the only link
we
social media
it
she
I
she
me
bad feelings
me
a short period
I
this festival
I
what
Jessie
It
me
the towel
I
the festival
it
this viewpoint
more time
we
I
her last personal email
me
it
we
it
work reasons
I
I
anything
us
she
the first moves
I
a friend
I
her
I
she
me
I
her
we
it
me
all these things
my head
I
that butch
I
a date
last night
I
a friend
her
a romantic one
I
it
I
butche

In [62]:
model = Model(text)

In [63]:
for ent in model.doc.ents:
    print(ent.label_)

DATE
DATE
CARDINAL
CARDINAL
PERSON
CARDINAL
ORG
CARDINAL
PERSON
CARDINAL
PERSON
ORDINAL
TIME
PERSON
DATE


In [64]:
for chunk in model.doc.noun_chunks:
    print(chunk.label_)

NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP
NP


In [68]:
cluster_n = 0
for cluster in model.doc._.coref_clusters:
    print(cluster_n, 'main:', cluster.main, cluster.main.root.ent_iob)
    for mention in cluster.mentions:
        print(cluster_n, mention, mention.root.ent_iob)
    cluster_n += 1

0 main: we 2
0 we 2
0 our 2
1 main: my friend 2
1 my friend 2
1 she 2
1 she 2
1 her 2
1 She 2
1 she 2
1 she 2
1 she 2
1 She 2
1 she 2
1 she 2
2 main: a cute new collar tho 2
2 a cute new collar tho 2
2 it 2
2 it 2
3 main: We 2
3 We 2
3 our 2
3 we 2
4 main: three more trays 2
4 three more trays 2
4 the trays 2
5 main: they 2
5 they 2
5 they 2
6 main: Jessie 3
6 Jessie 3
6 Jessie 3
6 she 2
6 she 2
6 she 2
6 her 2
6 she 2
6 her 2
6 she 2
6 her 2
6 her 2
6 her 2
7 main: three artists 2
7 three artists 2
7 them 2
8 main: our 2
8 our 2
8 we 2
8 we 2
8 us 2
9 main: me 2
9 me 2
9 me 2
9 me 2
10 main: this festival 2
10 this festival 2
10 the festival 2
11 main: It 2
11 It 2
11 it 2
12 main: them 2
12 them 2
12 them 2
13 main: this 2
13 this 2
13 it 2


In [44]:
model.doc[9:12].merge()

herself did not

In [45]:
print(model.doc[9], model.doc[9].pos_, model.doc[9].tag_)

herself did not PRON PRP


In [194]:
for noun_chunk in model.noun_chunks:
    print(noun_chunk)

I
a good day
I
we
the other half
our funding
my travel grant
my friend
me
she
I
some money
I
my dogs
the pet store
my girl dog
a new collar
she
everyone
an ongoing issue
her
She
she
she
the jugular
everyone
she
She
a cute new collar tho
it
pineapples
it
I
the dentist
she
my Invisalign progress
We
three more trays
she
an impression
my teeth
they
they
the rest
the trays
I
another payment
the end
my treatment
I
some work emails
the festival
Jessie
some important points
our potential artists
Mutual Friend
names
some other people
we
I
three artists
Jessie
the idea
them
it
I
she
I
our collective
It
I
the only link
we
social media
it
she
I
she
me
bad feelings
me
a short period
I
this festival
I
what
Jessie
It
me
the towel
I
the festival
it
this viewpoint
more time
we
I
her last personal email
me
it
we
it
work reasons
I
I
anything
us
she
the first moves
I
a friend
I
her
I
she
me
I
her
we
it
me
all these things
my head
I
that butch
I
a date
last night
I
a friend
her
a romantic one
I
it
I
butche

In [195]:
for cluster in model.doc._.coref_clusters:
    for mention in cluster:
        print(mention)

we
our
my friend
she
she
her
She
she
she
she
She
she
she
a cute new collar tho
it
it
We
our
we
three more trays
the trays
they
they
Jessie
Jessie
she
she
she
her
she
her
she
her
her
her
three artists
them
our
we
we
us
me
me
me
this festival
the festival
It
it
them
them
this
it


In [46]:
for cluster in nlp("Today is the 10th anniversary of the day that Katy adopted me. It’s also the 4th anniversary of the day she left me. I’ve been wanting to pay tribute to her, but this week has been so frustrating with ups and downs and maybes. It took an episode of Life Below Zero to draw it out of me. For as long as I can remember, I wanted to be a mama. I got married young but wanted to wait until college was over and I was established in my career before we started a family. I began to have a nagging feeling right away, that I’d never have a baby. I thought I was going to die young. I prayed and prayed a selfish prayer that God would just let me have that experience before he took me. As you may or may not know, my marriage ended before children came along. I had one unplanned pregnancy after that, but it ended in disaster after only a few weeks. As those of you know who have had a baby in your womb but never got to hold her in your arms, there is a magical transformation that takes place that makes you a member of a new club. Tonight I am frustrated again. I’ve learned that the test that is required to determine my “disability status” will cost $800, and is not covered by my insurance. As I said, the ups and downs and maybes are taking their toll this week. When we got home, I warned Drew that I’m emotional and please don’t take it personally, and then plopped into my chair. He brought me dinner then put on Life Below Zero, where one of the men featured was calling to his dog and she wasn’t moving. I already had tears in my eyes and a lump in my throat, anticipating his loss. But as he reached into her house to touch her, she looked up at him and revealed a litter of puppies. I’ve been crying ever since. When Katy came along, she had big paws to fill. Magnum, my first dog, had been an absolute angel. He had the most gentle temperament, but he was also a fierce protector. It was years after losing him before I could think of getting another dog. But the time came, that I was ready for another dog. I was going to be moving into a house by myself and wanted a medium sized dog as an early warning alarm if nothing else. The woman who was trying to find a home for her told me that she’d been abandoned at a trailer park. She was tied to a trailer where she was fed scraps by various park residents. That’s why she was so fat. The woman brought Katy to my apartment to meet me. I sat down on the floor across the room, and said, “Hi, Katy!” She walked straight to me and laid her head in my lap. She cuddled me like it was a familiar place for her. I think we both knew she was home. Katy was ravenous that first week or so. She’d ransack the garbage at night and she’d inhale her food. I’d accidentally left out a log of premium wet dog food. She had eaten half of it by the time I realized it. I couldn’t understand why she was so hungry, but I understood why she was so fat now. Or so I thought. One day I came home from work and found her laying in the kitchen with her head propped against a cabinet. I suddenly suspected that she wasn’t just fat, she might be pregnant! I got down on the floor with her and put my hands on her belly. I could feel the babies moving around!")._.coref_clusters:
    for mention in cluster.mentions:
        print(mention.text, '-', [token.text for token in mention.root.subtree])

the 10th anniversary of the day that Katy adopted me - ['the', '10th', 'anniversary', 'of', 'the', 'day', 'that', 'Katy', 'adopted', 'me']
It - ['It']
Katy - ['Katy']
she - ['she']
her - ['her']
this week - ['this', 'week']
It - ['It']
an episode of Life Below Zero - ['an', 'episode', 'of', 'Life', 'Below', 'Zero']
it - ['it']
God - ['God']
he - ['he']
one unplanned pregnancy - ['one', 'unplanned', 'pregnancy']
it - ['it']
a baby - ['a', 'baby']
her - ['her']
the ups and downs and maybes - ['the', 'ups', 'and', 'downs', 'and', 'maybes']
their - ['their']
He - ['He']
his - ['his']
his - ['his']
he - ['he']
him - ['him']
He - ['He']
he - ['he']
him - ['him']
myself - ['myself']
his dog - ['his', 'dog']
she - ['she']
her - ['her']
her - ['her']
she - ['she']
Katy - ['Katy']
she - ['she']
The woman who was trying to find a home for her - ['The', 'woman', 'who', 'was', 'trying', 'to', 'find', 'a', 'home', 'for', 'her']
her - ['her']
she - ['she']
She - ['She']
she - ['she']
she - ['she']
Th

In [47]:
model.resolved_text

"SHEILA was run over by a truck. SHEILA did not see that coming. USER told SHEILA SHEILA should take care of SHEILA, but USER know SHEILA will just go and do SHEILA's thing regardless of what USER say. What a conundrum! This makes USER wish USER had never signed up to be friends with SHEILA, although USER do love SHEILA."

In [48]:
model.doc._.coref_resolved

'Sheila was run over by a truck. Sheila did not see that coming. I told Sheila Sheila should take care of Sheila, but I know Sheila will just go and do Sheila thing regardless of what I say. What a conundrum! This makes me wish I had never signed up to be friends with Sheila, although I do love Sheila.'

In [98]:
for token in model.doc:
    print('-' + token.text_with_ws + '-')

-Sheila -
-was -
-run -
-over -
-by -
-a -
-truck-
-. -
-She -
-herself -
-did -
-not -
-see -
-that -
-coming-
-. -
-I -
-told -
-her -
-she -
-should -
-take -
-care -
-of -
-herself-
-, -
-but -
-I -
-know -
-she -
-will -
-just -
-go -
-and -
-do -
-her -
-thing -
-regardless -
-of -
-what -
-I -
-say-
-. -
-What -
-a -
-conundrum-
-! -
-This -
-makes -
-me -
-wish -
-I -
-had -
-never -
-signed -
-up -
-to -
-be -
-friends -
-with -
-her-
-, -
-although -
-I -
-do -
-love -
-the -
-girl-
-.-
-Sheila -
-was -
-run -
-over -
-by -
-a -
-truck-
-. -
-She -
-herself -
-did -
-not -
-see -
-that -
-coming-
-. -
-I -
-told -
-her -
-she -
-should -
-take -
-care -
-of -
-herself-
-, -
-but -
-I -
-know -
-she -
-will -
-just -
-go -
-and -
-do -
-her -
-thing -
-regardless -
-of -
-what -
-I -
-say-
-. -
-What -
-a -
-conundrum-
-! -
-This -
-makes -
-me -
-wish -
-I -
-had -
-never -
-signed -
-up -
-to -
-be -
-friends -
-with -
-her-
-, -
-although -
-I -
-do -
-love -
-the -
-girl-


-know -
-she -
-will -
-just -
-go -
-and -
-do -
-her -
-thing -
-regardless -
-of -
-what -
-I -
-say-
-. -
-What -
-a -
-conundrum-
-! -
-This -
-makes -
-me -
-wish -
-I -
-had -
-never -
-signed -
-up -
-to -
-be -
-friends -
-with -
-her-
-, -
-although -
-I -
-do -
-love -
-the -
-girl-
-.-
-Sheila -
-was -
-run -
-over -
-by -
-a -
-truck-
-. -
-She -
-herself -
-did -
-not -
-see -
-that -
-coming-
-. -
-I -
-told -
-her -
-she -
-should -
-take -
-care -
-of -
-herself-
-, -
-but -
-I -
-know -
-she -
-will -
-just -
-go -
-and -
-do -
-her -
-thing -
-regardless -
-of -
-what -
-I -
-say-
-. -
-What -
-a -
-conundrum-
-! -
-This -
-makes -
-me -
-wish -
-I -
-had -
-never -
-signed -
-up -
-to -
-be -
-friends -
-with -
-her-
-, -
-although -
-I -
-do -
-love -
-the -
-girl-
-.-
-Sheila -
-was -
-run -
-over -
-by -
-a -
-truck-
-. -
-She -
-herself -
-did -
-not -
-see -
-that -
-coming-
-. -
-I -
-told -
-her -
-she -
-should -
-take -
-care -
-of -
-herself-
-, -
-but -
-

-her-
-, -
-although -
-I -
-do -
-love -
-the -
-girl-
-.-
-Sheila -
-was -
-run -
-over -
-by -
-a -
-truck-
-. -
-She -
-herself -
-did -
-not -
-see -
-that -
-coming-
-. -
-I -
-told -
-her -
-she -
-should -
-take -
-care -
-of -
-herself-
-, -
-but -
-I -
-know -
-she -
-will -
-just -
-go -
-and -
-do -
-her -
-thing -
-regardless -
-of -
-what -
-I -
-say-
-. -
-What -
-a -
-conundrum-
-! -
-This -
-makes -
-me -
-wish -
-I -
-had -
-never -
-signed -
-up -
-to -
-be -
-friends -
-with -
-her-
-, -
-although -
-I -
-do -
-love -
-the -
-girl-
-.-
-Sheila -
-was -
-run -
-over -
-by -
-a -
-truck-
-. -
-She -
-herself -
-did -
-not -
-see -
-that -
-coming-
-. -
-I -
-told -
-her -
-she -
-should -
-take -
-care -
-of -
-herself-
-, -
-but -
-I -
-know -
-she -
-will -
-just -
-go -
-and -
-do -
-her -
-thing -
-regardless -
-of -
-what -
-I -
-say-
-. -
-What -
-a -
-conundrum-
-! -
-This -
-makes -
-me -
-wish -
-I -
-had -
-never -
-signed -
-up -
-to -
-be -
-friends -
-wit

NameError: name 'doc' is not defined

In [40]:
len(model.people)

1

In [41]:
[person.refs for person in model.people]

[[She herself, the girl, her, she, her, she, her, herself, She, Sheila]]

In [42]:
for person in model.people:
    for ref, res in zip(person.refs, person.resolved_refs):
        print(ref.start, ref, res.start, res)
#     print(person.name, person.refs, person.resolved_refs)

8 She herself 8 Sheila
66 the girl 65 love
60 her 58 friends
19 she 17 Sheila
18 her 16 told
29 she 27 know
35 her 33 do
24 herself 22 of
8 She 6 truck
0 Sheila 66 Sheila


In [13]:
print(len([token for token in model.doc]), len([token for token in model.resolved_doc]))

69 68


In [14]:
text = 'November was a trying month… on the 7th Dante had a major accident. 5 minutes before school and he and some friends are climbing the fence, I tell him it’s not a good idea and to get down. I turn back to talk to Jodi (on of my best mom friend’s at the school) and Dante comes to me screaming with his hand full of blood. I run him into my classroom and get him to the sink, as I turn on the water to clean the area the flap of his thumb lifts away and I see the bone. Shit. This isn’t something I can fix here, I grab my first aid kit and wrap it like crazy because it’s bleeding like crazy. I phone James and tell him to get to the ER as Dante is screaming and freaking out in the background as I’m trying to usher him back to the car as he’s bleeding like a stuffed pig. Unfortunately in the ER I learned that my child doesn’t take to freezing, an hour of gel freezing and he still felt the 2 needles as they went in, 15 minutes later and he felt the last 2 stitches of 8. He needed more because his finger still had gaps, the doctor didn’t want to cause him anymore pain so he glued them. It was an intense and deep gash that spiraled all the way up his thumb. I was trying to stay strong for him but I did break down as he screamed and cried, I was left to emotionally drained that day. James was able to take the remainder of the day off and stay with him. He missed 2 more days of school and then had an extra long weekend due to the holiday and the pro day but for 2 weeks he couldn’t write (of course it was his right hand.) 3 doctor visits later and he finally got them out full last week, the first visit the doctor wanted them in longer because of the severity. 2nd time he could only get 6 out because the glue had gotten on the last 2 stitches and he didn’t want to have to dig them out so we had to soak and dissolve the glue for 3 days. 3rd time the last 2 came out.  Even now he’s slowly regaining his writing skills as there was some nerve damage.'

In [9]:
adoc = nlp(text)
# for cluster in adoc._.coref_clusters:
#     for mention in cluster.mentions:
#         mention.merge()
        
for cluster in adoc._.coref_clusters:
    for mention in cluster.mentions:
        print(cluster)

the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
school: [school, the school]
school: [school, the school]
my first aid kit: [my first aid kit, it, it]
my first aid kit: [my first aid kit, it, it]
my first aid kit: [my first aid kit, it, it]


IndexError: [E037] Error calculating span: Can't find a token ending at character offset 1001.

In [10]:
adoc = nlp(text)
# for cluster in adoc._.coref_clusters:
#     for mention in cluster.mentions:
#         mention.merge()
        
for cluster in adoc._.coref_clusters:
    for mention in cluster.mentions:
        print(cluster)

the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
school: [school, the school]
school: [school, the school]
my first aid kit: [my first aid kit, it, it]
my first aid kit: [my first aid kit, it, it]
my first aid kit: [my first aid kit, it, it]
James: [James, him, him, he, he, he, He, his, him, he, his, him, he]
James: [James, him, him, he, he, he, He, his, him, he, his, him, he]
James: [James, him, him, he, he, he, He, his, him, he, his, him, he]
James: [James, him, him, he, he, he, He, his, him, he, his, him,

In [71]:
assert 1==2

AssertionError: 

In [73]:
for token in model.doc:
    print(token, token.ent_iob)

So 2
I 2
have 2
had 2
a 3
good 1
day 1
today 3
. 2
I 2
found 2
out 2
we 2
got 2
the 3
other 1
half 1
of 2
our 2
funding 2
for 2
my 2
travel 2
grant 2
, 2
which 2
paid 2
for 2
my 2
friend 2
to 2
come 2
with 2
me 2
. 2
So 2
that 2
's 2
good 2
, 2
she 2
and 2
I 2
will 2
both 2
get 2
some 2
money 2
back 2
. 2
I 2
took 2
my 2
dogs 2
to 2
the 2
pet 2
store 2
so 2
my 2
girl 2
dog 2
could 2
get 2
a 2
new 2
collar 2
, 2
but 2
she 2
wanted 2
to 2
beat 2
everyone 2
up 2
. 2
This 2
is 2
an 2
ongoing 2
issue 2
with 2
her 2
. 2
She 2
's 2
so 2
little 2
and 2
cute 2
too 2
but 2
damn 2
she 2
acts 2
like 2
she 2
's 2
gon 2
na 2
go 2
for 2
the 2
jugular 2
with 2
everyone 2
she 2
does 2
not 2
know 2
! 2
She 2
did 2
end 2
up 2
with 2
a 2
cute 2
new 2
collar 2
tho 2
, 2
it 2
has 2
pineapples 2
on 2
it 2
. 2
I 2
went 2
to 2
the 2
dentist 2
and 2
she 2
's 2
happy 2
with 2
my 2
Invisalign 2
progress 2
. 2
We 2
have 2
three 3
more 2
trays 2
and 2
then 2
she 2
does 2
an 2
impression 2
to 2
make 2
sure 2
my 2
te

In [104]:
model.doc[0:1].label_

''

In [105]:
model.doc._.coref_clusters[0].mentions[0].label_

''

In [148]:
for chunk in model.doc.noun_chunks:
    print(chunk.root, chunk.root.tag_, chunk.root.pos_)

I PRP PRON
day NN NOUN
I PRP PRON
we PRP PRON
half NN NOUN
funding NN NOUN
grant NN NOUN
friend NN NOUN
me PRP PRON
she PRP PRON
I PRP PRON
money NN NOUN
I PRP PRON
dogs NNS NOUN
store NN NOUN
dog NN NOUN
collar NN NOUN
she PRP PRON
everyone NN NOUN
issue NN NOUN
her PRP PRON
She PRP PRON
she PRP PRON
she PRP PRON
jugular NN NOUN
everyone NN NOUN
she PRP PRON
She PRP PRON
tho NN NOUN
it PRP PRON
pineapples NNS NOUN
it PRP PRON
I PRP PRON
dentist NN NOUN
she PRP PRON
progress NN NOUN
We PRP PRON
trays NNS NOUN
she PRP PRON
impression NN NOUN
teeth NNS NOUN
they PRP PRON
they PRP PRON
rest NN NOUN
trays NNS NOUN
I PRP PRON
payment NN NOUN
end NN NOUN
treatment NN NOUN
I PRP PRON
emails NNS NOUN
festival NN NOUN
Jessie NNP PROPN
points NNS NOUN
artists NNS NOUN
Friend NNP PROPN
names NNS NOUN
people NNS NOUN
we PRP PRON
I PRP PRON
artists NNS NOUN
Jessie NNP PROPN
idea NN NOUN
them PRP PRON
it PRP PRON
I PRP PRON
she PRP PRON
I PRP PRON
collective NN NOUN
It PRP PRON
I PRP PRON
link NN NO

In [150]:
for ent in model.doc.ents:
    print(ent, ent.root, ent.root.tag_, ent.root.pos_)

a good day day NN NOUN
today today NN NOUN
the other half half NN NOUN
three three CD NUM
Jessie Jessie NNP PROPN
one one CD NUM
Mutual Friend Friend NNP PROPN
three three CD NUM
Jessie Jessie NNP PROPN
one one CD NUM
Jessie Jessie NNP PROPN
first first JJ ADJ
last night night NN NOUN
Femmes Femmes NNS NOUN
January January NNP PROPN


In [ ]:
doc2 = nlp('I need to go to the dentist')

